In [2]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.9.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.9.2


In [3]:
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 29 kB/s 
     |████████████████████████████████| 3.8 MB 59.8 MB/s 
     |████████████████████████████████| 503 kB 63.6 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6f53da17dd34b3b78ec54bfcfda482f2f62d62f5d7b0e5025f6aa5d648c4f6fc
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

In [15]:
import tensorflow as tf
import numpy as np

# [털, 날개] [0, 0] = 기타 / [1, 0] = 포유류 / [_, 1] = 조류 
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1], # 기타, 포유류, 조류, 기타, 기타, 조류
                   [1, 0], [0, 1], [0, 1], [1, 0], [1, 1], [0, 1], # 포유류, 조류, 조류, 포유류, 조류, 조류
                   [1, 0], [1, 1], [0, 1], [1, 1], [0, 0], [0, 1], # 포유류, 조류, 조류, 조류, 기타, 조류
                   [0, 0], [1, 0], [0, 0], [0, 1], [1, 0], [1, 1]]) # 기타, 포유류, 기타, 조류, 포유류, 조류

# 기타 [1, 0, 0], 포유류 [0, 1, 0], 조류 [0, 0, 1]
y_data = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], # 기타, 포유류, 조류, 기타, 기타, 조류
                   [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], # 포유류, 조류, 조류, 포유류, 조류, 조류
                   [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], # 포유류, 조류, 조류, 조류, 기타, 조류
                   [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1]]) # 기타, 포유류, 기타, 조류, 포유류, 조류

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 가중치 편향 추가
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([20]))
b3 = tf.Variable(tf.zeros([3]))

# 특징 입력값에 첫 번째 가중치와 편향, 활성화 함수 적용
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 두 번째 가중치와 편향, 활성화 함수 적용
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.relu(L2)

# 세 번째 가중치와 편향을 적용하여 최종 모델 생성
model = tf.add(tf.matmul(L2, W3), b3)

In [22]:
# 손실함수 : cross entropy / 최적화 함수 : AdamOptimizer >> Gradient Descent Optiomization 중 하나
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print('학습횟수|손실값')
# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(30):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  # 학습 도중 10번에 한 번씩 손실값을 출력해 봅니다.
  if (step + 1 ) % 10 == 0:
    print(step + 1, '\t|', sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 학습 결과 확인
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 정확도 출력
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

학습횟수|손실값
10 	| 0.5489585
20 	| 0.32364836
30 	| 0.16462947
예측값: [0 1 2 0 0 2 1 2 2 1 2 2 1 2 2 2 0 2 0 1 0 2 1 2]
실제값: [0 1 2 0 0 2 1 2 2 1 2 2 1 2 2 2 0 2 0 1 0 2 1 2]
정확도: 100.00
